<a href="https://colab.research.google.com/github/YunaW-Dev/-TensorFlow-KaggleCompetition-CatVsDog/blob/main/machine%20learning%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!wget https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip

--2021-11-05 15:36:20--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.193.128, 172.217.204.128, 172.217.203.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.193.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip.1’

cats_and_dogs_filte 100%[===================>]  65.43M   136MB/s    in 0.5s    

2021-11-05 15:36:21 (136 MB/s) - ‘cats_and_dogs_filtered.zip.1’ saved [68606236/68606236]



In [27]:
import zipfile

zip_dir= 'cats_and_dogs_filtered.zip'
zip =zipfile.ZipFile(zip_dir, 'r')
zip.extractall()
zip.close()

In [28]:
import os
base_dir= 'cats_and_dogs_filtered'
train_dir= os.path.join(base_dir, 'train')
validation_dir= os.path.join(base_dir, 'validation')
train_cats_dir= os.path.join(train_dir, 'cats')
train_dogs_dir= os.path.join(train_dir, 'dogs')
validation_cats_dir=os.path.join(validation_dir, 'cats')
validation_dogs_dir=os.path.join(validation_dir, 'dogs')

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_data=ImageDataGenerator( rescale = 1.0/255 )
test_data=ImageDataGenerator( rescale = 1.0/255 )
train_generator=train_data.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    class_mode='binary',
                                                    target_size=(150, 150))     
validation_generator=test_data.flow_from_directory(validation_dir,
                                                         batch_size=20,
                                                         class_mode  = 'binary',
                                                         target_size = (150, 150))

In [29]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2), 
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(1024, activation='relu'), 
    tf.keras.layers.Dense(1, activation='sigmoid')  
])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 17, 17, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 18496)            

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epochs, logs={}):
            if (logs.get('acc')>0.98):
                self.model.stop_training = True

train_result = model.fit(train_generator,
                              validation_data=validation_generator,
                              steps_per_epoch=100,
                              epochs=15,
                              validation_steps=50,
                              verbose=2)